In [126]:
from datasets import list_datasets, load_dataset
from pprint import pprint
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pickle

from os import listdir
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from nltk.corpus import stopwords
import sys


from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [127]:
path_of_downloaded_files = "/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/Natural Language Processing/glove.6B.50d.txt"
filename = "glove.6B.50d.txt"

SOS_token = 0
EOS_token = 1

sos = 'sos'
eos = 'eos'

embedding_dim = hidden_size = 50
data_size = 10000
MAX_LENGTH = 99

In [128]:
# glove_file = datapath(path_of_downloaded_files)
# word2vec_glove_file = get_tmpfile(filename)
# glove2word2vec(glove_file, word2vec_glove_file)
# word_vectors = KeyedVectors.load_word2vec_format(word2vec_glove_file)
# print('loaded %s word vectors from %s.' % (len(word_vectors.key_to_index),filename ))

In [129]:
train_dataset = load_dataset('ccdv/cnn_dailymail','3.0.0',split = 'train')  
test_dataset = load_dataset('ccdv/cnn_dailymail','3.0.0',split = 'test')  
validation_dataset = load_dataset('ccdv/cnn_dailymail','3.0.0',split = 'validation')  

entire_dataset = load_dataset('ccdv/cnn_dailymail', '3.0.0')

print(train_dataset.__len__())
print(test_dataset.__len__())
print(validation_dataset.__len__())
print(entire_dataset.__len__())

Reusing dataset cnn_dailymail (/Users/rohankilledar/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f)
Reusing dataset cnn_dailymail (/Users/rohankilledar/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f)
Reusing dataset cnn_dailymail (/Users/rohankilledar/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f)
Reusing dataset cnn_dailymail (/Users/rohankilledar/.cache/huggingface/datasets/ccdv___cnn_dailymail/3.0.0/3.0.0/0107f7388b5c6fae455a5661bcd134fc22da53ea75852027040d8d1e997f101f)
100%|██████████| 3/3 [00:00<00:00, 41.79it/s]

287113
11490
13368
3


In [130]:
from torchtext.vocab import GloVe
embedding_glove = GloVe(name='6B', dim=50)

In [131]:
dataset = {
    'train' : train_dataset,
    'test' : test_dataset,
    'validation' : validation_dataset
}

with open("hf_dataset.pkl","wb") as save_path:
    pickle.dump(dataset, save_path)

# with open('hf_dataset.pkl','rb') as ff:
#     hfd = pickle.load(ff)

# train_ds = hfd['train']
# train_df = train_ds.data.to_pandas()
# test_ds = hfd['test']
# test_df = test_ds.data.to_pandas()
# valid_ds = hfd['validation']
# valid_df = valid_ds.data.to_pandas()


In [132]:
# train_df = train_dataset.data.to_pandas()
# train_df = pd.DataFrame.from_dict(train_dataset)
train_df = train_dataset.data.to_pandas()
test_df = test_dataset.data.to_pandas()
valid_df = validation_dataset.data.to_pandas()
dataset_list = [train_df,test_df,valid_df]

In [133]:
train_df.head()

,article,highlights,id
0,It's official: U.S. President Barack Obama wan...,Syrian official: Obama climbed to the top of t...,0001d1afc246a7964130f43ae940af6bc6c57f01
1,(CNN) -- Usain Bolt rounded off the world cham...,Usain Bolt wins third gold of world championsh...,0002095e55fcbd3a2f366d9bf92a95433dc305ef
2,"Kansas City, Missouri (CNN) -- The General Ser...",The employee in agency's Kansas City office is...,00027e965c8264c35cc1bc55556db388da82b07f
3,Los Angeles (CNN) -- A medical doctor in Vanco...,NEW: A Canadian doctor says she was part of a ...,0002c17436637c4fe1837c935c04de47adb18e9a
4,(CNN) -- Police arrested another teen Thursday...,Another arrest made in gang rape outside Calif...,0003ad6ef0c37534f80b55b4235108024b407f0b


In [134]:
# train_reduced = train_dataset[:2000]
# train_reduced = pd.DataFrame.from_dict(train_reduced)
# train_reduced['article'] = train_reduced['article'].apply(lambda x: normalizeString(x))
# train_reduced['highlights'] = train_reduced['highlights'].apply(lambda x: normalizeString(x))
# train_reduced.drop('id', axis=1, inplace=True)
# train_reduced.head()

# train_reduced = train_reduced[train_reduced['article'].str.split().str.len().lt(mean_article_len)]


In [135]:
def decontracted(phrase): 
    phrase = re.sub(r"won't", "will not", phrase) 
    phrase = re.sub(r"can\'t", "can not", phrase)  
    phrase = re.sub(r"n\'t", " not", phrase)  
    phrase = re.sub(r"\'re", " are", phrase)  
    phrase = re.sub(r"\'s", " is", phrase)  
    phrase = re.sub(r"\'d", " would", phrase)  
    phrase = re.sub(r"\'ll", " will", phrase)  
    phrase = re.sub(r"\'t", " not", phrase)  
    phrase = re.sub(r"\'ve", " have", phrase)  
    phrase = re.sub(r"\'m", " am", phrase)  
    phrase = re.sub(r'ain\'t', "is not", phrase)
    
    return phrase

In [136]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    
    stop_words = stopwords.words('english')
    un = '(CNN)'
    dash_indx = s.find(un)
    if dash_indx>-1: #and dash_indx<=20:
        s = s[dash_indx+len(un):]
    
    s = s.lower().strip()
    s=decontracted(s)

    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r'\.', r' . ', s)
    

    # s = ' '.join([word for word in s.split(' ') if word not in stop_words and word in vocab.word2index.keys()])
  
    return s

In [137]:
import re
def drop_data_id(dataset_list):
    for data in dataset_list:
        data.drop('id', axis=1, inplace=True)

drop_data_id(dataset_list)



In [138]:
class Lang(object):
    def __init__(self, name):
        self.name = name
        self.word2index = {'sos': 0 , 'eos': 1}
        self.word2count = {'sos': 1, 'eos': 1}
        self.index2word = {0: "sos", 1: "eos"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in regTokenize(sentence):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
    
    def addFromPretrained(self,emb):
        for i in range(len(embedding_glove)):
            word = embedding_glove.itos[i]
            if word not in self.word2index:
                self.word2index[word] = self.n_words
                self.word2count[word] = 1
                self.index2word[self.n_words] = word
                self.n_words +=1
            

# vocab = Lang('vocab')
# vocab.addFromPretrained(embedding_glove)   

In [139]:
train_df['article'] = train_df['article'].apply(lambda x: normalizeString(x))
train_df['highlights'] = train_df['highlights'].apply(lambda x: normalizeString(x))
# test_df['article'] = test_df['article'].apply(lambda x: normalizeString(x))
# test_df['highlights'] = test_df['highlights'].apply(lambda x: normalizeString(x))
# valid_df['article'] = valid_df['article'].apply(lambda x: normalizeString(x))
# valid_df['highlights'] = valid_df['highlights'].apply(lambda x: normalizeString(x))


In [140]:
train_df['word_count_text'] = train_df['article'].apply(lambda x: len(str(x).split()))
train_df['highlight_count'] = train_df['highlights'].apply(lambda x: len(str(x).split()))
from math import floor

mean_article_len = floor(train_df['word_count_text'].mean())
mean_hightlight_len = floor(train_df['highlight_count'].mean())

max_article_len = floor(train_df['word_count_text'].max())
max_hightlight_len = floor(train_df['highlight_count'].max())



print("The mean word count length of text article is " + str(mean_article_len))
print("The mean word count length of summary/highlight is " + str(mean_hightlight_len))


print("The max word count length of text article is " + str(max_article_len))
print("The max word count length of summary/highlight is " + str(max_hightlight_len))

train_df.drop('word_count_text', axis=1, inplace=True)
train_df.drop('highlight_count', axis=1, inplace=True)

The mean word count length of text article is 727
The mean word count length of summary/highlight is 52
The max word count length of text article is 2785
The max word count length of summary/highlight is 1278


In [141]:
with open("processed_train_df.DataFrame","wb") as save_path:
    pickle.dump(train_df, save_path)

In [142]:
#using this tokenizer as its the fastest way to tokenize sentences.
WORD = re.compile(r'\w+')
def regTokenize(text):
    words = WORD.findall(text)
    return words

In [143]:
# class Lang(object):
#     def __init__(self,vocab_file) -> None:
#         self.word2index = {}
#         self.index2word = {}

#         index = 0
#         for word in [pad,unk,sos, eos]:
#             self.word2index[word] = index
#             self.index2word[index] = word
#             index +=1

#         with open(vocab_file, 'r') as f:
#             for line in f:
#                 try:
#                     word = line.split()[0]
#                     self.word2index[word] = index
#                     self.index2word[index] = word
#                     index +=1

#                 except ValueError as e:
#                     print("Error in line {}: {}".format(token_counter - 4, e))

#     def word2index(self,word: str) -> int:
#         if word == eos:
#             return self.word2index['.']
#         elif word not in self.word2index:
#             return self.word2index[]

# new_vocab = Lang(path_of_downloaded_files)
# print(len(new_vocab.word2index))

In [144]:
# class Lang:
#     def __init__(self, name):
#         self.name = name
#         self.words = []
#         self.word2index = {}
#         self.word2count = {}
#         self.index2word = {}
#         self.n_words = 0  # Count SOS and EOS
#         self.glove = {}
#         self.embedding_dim = word_vectors.get_vector('office').shape[0]

#     def addSentence(self, sentence):
#         for word in regTokenize(sentence):
#             self.addWord(word)

#     def addWord(self, word):
#         if word not in self.words:
#             self.words.append(word)
#             self.word2index[word] = self.n_words
#             self.word2count[word] = 1
#             self.index2word[self.n_words] = word
#             self.n_words += 1
#             if word not in self.glove:
#                 self.glove[word] = np.random.normal(scale=0.6 , size= (self.embedding_dim,))
#         else:
#             self.word2count[word] += 1

#     def addPretrained(self,path_of_downloaded_files):
#         with open(path_of_downloaded_files) as f:
#             for indx,l in tqdm(enumerate(f)):
#                 line = l.split()
#                 word = line[0]
#                 if word == sos:
#                     first_word = self.words[0]
#                     self.words.append(first_word)
#                     self.words[0] = sos
#                     self.word2index[first_word] = indx
#                     self.word2index[sos] = 0
#                     self.index2word[indx] = first_word
#                     self.index2word[0] = sos
#                     self.glove[sos] = word_vectors[indx]
#                     #self.glove[first_word] = word_vectors[0]
#                     self.word2count[sos] = 1
#                     self.n_words += 1

#                 elif word == eos:
#                     sec_word = self.words[1]
#                     self.words.append(sec_word)
#                     self.words[1] = eos
#                     self.index2word[indx] = sec_word
#                     self.index2word[1] = eos
#                     self.word2index[sec_word] = indx
#                     self.word2index[eos] = 1
#                     self.glove[eos] = word_vectors[indx]
#                     #self.glove[sec_word] = word_vectors[1]
#                     self.word2count[eos] = 1
#                     self.n_words += 1
#                 else:
#                     self.words.append(word)
#                     self.word2count[word] = 1
#                     self.word2index[word] = indx
#                     self.index2word[indx]=word
#                     self.glove[word] = word_vectors[indx]
#                     self.n_words += 1
                    

In [145]:
vocab = Lang('vocab')
vocab.addFromPretrained(path_of_downloaded_files)
def prepareData(vocab):
    for _ ,row in tqdm(train_df.iterrows()):
        vocab.addSentence(row['article'])
        vocab.addSentence(row['highlights'])
    print("counted Words:")
    print(vocab.name, vocab.n_words)
    
prepareData(vocab)

with open("vocab.pkl","wb") as save_path:
    pickle.dump(vocab, save_path)

0it [00:00, ?it/s]

counted Words:
vocab 658217


In [146]:
with open("vocab.pkl","wb") as save_path:
    pickle.dump(vocab, save_path)

In [147]:
pairs = train_df.values.tolist()


In [148]:
def filter_pair(data, max_len):
    return len(data[0].split(' ')) < max_len

def filter_pairs(pairs, max_len):
    return [pair for pair in pairs if filter_pair(pair,max_len)]



In [149]:
pairs = filter_pairs(pairs, mean_article_len)

In [150]:
def indexesFromSentences(lang, sentence):
    return [lang.word2index[word] for word in regTokenize(sentence)]

def tensorFromSentences(lang, sentence):
    indexes = indexesFromSentences(lang,sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype = torch.long).view(-1,1)

def tensorFromPair(pair):
    input_tensor = tensorFromSentences(vocab, pair[0])
    target_tensor = tensorFromSentences(vocab, pair[1])
    return (input_tensor, target_tensor)

In [151]:

def generate_weights(unique_word_corpus, tens=True):
    matrix_len = len(unique_word_corpus)
    
    
    weight_matrix  = np.zeros((matrix_len, embedding_dim))
   
    words_found = 0
    

    for indx, word in enumerate(unique_word_corpus):
        # try:
            # if word is not in the embedding this function assign zero value to that vector
        weight_matrix[indx] = embedding_glove.get_vecs_by_tokens(word)
        words_found +=1

        # except KeyError:

        #     weight_matrix[indx] = np.random.normal(scale=0.6 , size= (embedding_dim,))
        #     random_weights +=1
    print(f'total words processed: {words_found}')
    # print(f'embedding initialized to random value for {random_weights} no. of words')
    if tens:
        return torch.from_numpy(weight_matrix) 
    else:
        return weight_matrix

weight_matrix= generate_weights(vocab.word2index.keys())

total words processed: 658217


In [152]:
weight_matrix = weight_matrix.to(torch.float32)

In [153]:

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
       

       # self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size,hidden_size)
        self.embedding = self.embedding.from_pretrained(weight_matrix, freeze = True)
        
        ## trying gru
        # self.gru = nn.GRU(hidden_size,hidden_size)

        self.lstm = nn.LSTM(input_size = hidden_size, hidden_size = hidden_size) #, num_layers = 1, bidirectional = True, batch_first = True)
        #self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1,1,-1)
        output = embedded
        
        # output, hidden = self.gru(output, hidden)
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def initHidden(self):
        #in case of LSTM we need h0 and c0 hence init this as a tuple (h0,c0) and passed as hidden to lstm and in case of gru its just h0
        return (torch.zeros(1, 1, self.hidden_size, dtype=torch.float, device=device), torch.zeros(1,1,self.hidden_size,dtype = torch.float, device= device))
        # return torch.zeros(1,1,self.hidden_size, device=device)

# %%
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size


        self.embedding = nn.Embedding(num_embeddings= output_size,embedding_dim= hidden_size)
        # self.gru = nn.GRU(hidden_size,hidden_size)
        self.embedding = self.embedding.from_pretrained(weight_matrix, freeze = True)
        self.lstm = nn.LSTM(input_size= hidden_size, hidden_size= hidden_size) #, num_layers = 1, bidirectional = False, batch_first = True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim= 1)
        # self.softmax = nn.Softmax(dim = 1)
       # self.fc = nn.Linear(in_features= hidden_size , out_features=vocab.n_words)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1,1,-1)
        output = torch.tanh(embedded)
        # output, hidden = self.gru(output, hidden)
        output, hidden = self.lstm(output.float(), hidden)
        #output = self.softmax(self.out(output[0]))
        output = self.softmax(self.out(output[0]))
        # output = self.out(output[0])
        return output, hidden

    def initHiddden(self):
        return (torch.zeros(1, 1, self.hidden_size, dtype=torch.float, device=device), torch.zeros(1,1,self.hidden_size,dtype = torch.float, device= device))


# %%
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=max_article_len):
        super(AttnDecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.embedding.from_pretrained(weight_matrix)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
       
    
        AttnDecoderRNN.embedded = embedded
        
        AttnDecoderRNN.hidden = hidden
        AttnDecoderRNN.encoder_outputs = encoder_outputs

        
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0].view(1,-1)), 1)), dim=1)
        # print(attn_weights.unsqueeze(0).shape, encoder_outputs.unsqueeze(0).shape)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

                                 

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
       return (torch.zeros(1, 1, self.hidden_size, dtype=torch.int64, device=device), torch.zeros(1,1,self.hidden_size,dtype = torch.int64, device= device))


In [154]:

# %%
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, attn = True, max_length=max_article_len):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_article_len, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            if attn :
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
            else:
                decoder_output, decoder_hidden= decoder(
                    decoder_input, decoder_hidden)
                #tar_tens = weight_matrix[target_tensor[di].item()].view(-1).float()
                # target_tensor -> tensor(4,dtype=long)
            loss += criterion(decoder_output, target_tensor[di])
            
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            if attn :
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
            else:
                decoder_output, decoder_hidden= decoder(
                    decoder_input, decoder_hidden)
            
            # sm = nn.Softmax(decoder_output)
            topv, topi = decoder_output.data.topk(1)
            # topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            # tar -> tensor((4)) dtype = long/int64
            #dec -> size(1,vocab_size) dtype = float
            # tar_tens = weight_matrix[target_tensor[di].item()].view(-1).float()
            loss += criterion(decoder_output, target_tensor[di])
            # sys.stdout.write("\rtarget: %s" % vocab.index2word[target_tensor[di].item()])
            sys.stdout.write("\rdecoded: %s" % vocab.index2word[topi.item()])
            sys.stdout.flush()
            train.dout = decoder_output
            # print(f'target output: {summary_vocab.index2word[target_tensor[di].item()]}')
            # print(f'decoded output: {vocab.index2word[decoder_output.data.topk(1)[1].item()]} ')
            if decoder_input.item() == EOS_token:
                break
    

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [155]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [156]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [157]:

# %%
#try different learning rate
def trainIters(encoder, decoder, n_iters, attn=False, print_every=1000, plot_every=100, learning_rate=0.1):
    start = time.time()
    if attn:
        print("training with attention.")
    

    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    print('encoder optimizer init')
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    print('decoder optimizer init')
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    
    print('choosing training pairs:' + str(n_iters))
    training_pairs = [tensorFromPair(random.choice(pairs))
                      for _ in range(n_iters)]
    
    print(f'filtering the pairs with max article size of {mean_article_len}')
    
    
    print('init criterion')
    criterion = nn.NLLLoss()
    # criterion = nn.CrossEntropyLoss()

    
    print('starting iterations')
    
    for iter in tqdm(range(1, n_iters + 1)):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion, attn)
        
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)
    


In [158]:

# %%
def evaluate(encoder, decoder, sentence ,max_length=max_article_len, attn=True): # please check what should be max_length
    with torch.no_grad():
        input_tensor = tensorFromSentences(vocab, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_article_len, encoder.hidden_size, device=device)
        print('evaluation init')
        for ei in range(input_length):
            
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(99):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        # print(decoder_attention,decoder_attention.shape)
        decoder_attentions[di] = decoder_attention.data
        topv, topi = decoder_output.data.topk(1)
        if topi.item() == EOS_token:
            decoded_words.append(eos)
            break
        else:
            decoded_words.append(vocab.index2word[topi.item()])

        decoder_input = topi.squeeze().detach()

    return decoded_words, decoder_attentions[:di + 1]



In [159]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
     #   print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [161]:

# %%
hidden_size = 50
voc_size = len(vocab.word2index)
hidden_dim = 50
# max_summary_size = 60
encoder1 = EncoderRNN(vocab.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, vocab.n_words).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, vocab.n_words, dropout_p=0.1).to(device)


# %%
# trainIters(encoder1, decoder1, 300, print_every=10)

trainIters(encoder1, attn_decoder1,30000,True, print_every=250)



training with attention.
encoder optimizer init
decoder optimizer init
choosing training pairs:30000
filtering the pairs with max article size of 727
init criterion
starting iterations


  0%|          | 0/30000 [00:00<?, ?it/s]

decoded: the8m 19s (- 991m 24s) (250 0%) 9.4162
decoded: has16m 45s (- 988m 48s) (500 1%) 8.1918
decoded: eos24m 56s (- 972m 32s) (750 2%) 8.2894
decoded: eos33m 8s (- 960m 59s) (1000 3%) 8.1810
decoded: in41m 0s (- 943m 7s) (1250 4%) 7.3423
decoded: and49m 29s (- 940m 26s) (1500 5%) 7.7972
decoded: eos57m 39s (- 930m 53s) (1750 5%) 7.8183
decoded: the66m 46s (- 934m 51s) (2000 6%) 8.0862
decoded: with75m 7s (- 926m 28s) (2250 7%) 7.2223
decoded: the83m 49s (- 922m 1s) (2500 8%) 7.5007
decoded: eos92m 14s (- 914m 3s) (2750 9%) 7.5574
decoded: in99m 34s (- 896m 12s) (3000 10%) 6.8960
decoded: eos108m 17s (- 891m 18s) (3250 10%) 7.5619
decoded: his117m 15s (- 887m 48s) (3500 11%) 7.7724
decoded: eos125m 33s (- 878m 54s) (3750 12%) 7.5668
decoded: they133m 45s (- 869m 23s) (4000 13%) 7.6948
decoded: eos142m 38s (- 864m 16s) (4250 14%) 7.8455
decoded: is151m 11s (- 856m 47s) (4500 15%) 7.5405
decoded: to159m 19s (- 846m 57s) (4750 15%) 7.4208
decoded: a167m 57s (- 839m 48s) (5000 16%) 7.61

In [162]:
evaluateRandomly(encoder1, attn_decoder1)

= remix of justin bieber is u smile has more than  .  million plays on soundcloud  . com  .  producer nick pittsinger says he slowed down the song percent  .  mtv disagrees saying the song is actually an untitled song from the photon wave orchestra  . 
evaluation init
< the of the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the to the

= influential british rock band the stone roses to reform after year hiatus  .  music writer says group will want gigs to improve the stone roses legacy  .  paul stokes says older groups allow fans to experience better quality music  .  but he says reformed acts are in danger of soaking up much of the live market  . 
evaluation init
< the of the to the to the to the to the to th

In [ ]:
train.dout.data.topk(1)
vocab.index2word[9]

'a'